# 1. Встановлення та імпортування бібліотек, визначення констант

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import json
import subprocess
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

import matplotlib.pyplot as plt

from PIL import Image, ImageTk # for handling and displaying images
import tkinter as tk # for creating graphical user interfaces
import datetime # for getting current time
import cv2 # for computer vision tasks like image processing and video capture
import os # for interacting with the file system
import threading # for performing multiple tasks simultaneously
import time # for measuring time and delaying execution
from tkinter.scrolledtext import ScrolledText # for creating a text widget with scrollbars

frame_width = 1280
frame_height = 720
frame_rate = 30
num_of_frames = 90

actions = np.array(os.listdir('videos'))

label_map = {label:num for num, label in enumerate(actions)}

# 2. Створення набору даних

## 2.1 Створення словника

In [2]:
dictionary = {
    "apple(r)": "яблуко",
    "apple(l)": "яблуко",
    "ball": "м'яч",
    "bike": "велосипед",
    "book": "книга",
    "bowl": "миска",
    "boy(r)": "хлопчик",
    "boy(l)": "хлопчик",
    "brother(r)": "брат",
    "brother(l)": "брат",
    "butterfly": "метелик",
    "car(r)": "автомобіль",
    "car(l)": "автомобіль",
    "chess": "шахи",
    "day": "день",
    "evening(r)": "вечір",
    "evening(l)": "вечір",
    "father(r)": "батько",
    "father(l)": "батько",
    "friend(r)": "друг",
    "friend(l)": "друг",
    "fruit(r)": "фрукти",
    "fruit(l)": "фрукти",
    "good(r)": "добрий",
    "good(l)": "добрий",
    "goodbye(r)": "до побачення",
    "goodbye(l)": "до побачення",
    "guitar(r)": "гітара",
    "guitar(l)": "гітара",
    "hello(r)": "привіт",
    "hello(l)": "привіт",
    "home": "дім",
    "I(r)": "я",
    "I(l)": "я",
    "interpreter": "перекладач",
    "island(r)": "острів",
    "island(l)": "острів",
    "medal(r)": "медаль",
    "medal(l)": "медаль",
    "morning(r)": "ранок",
    "morning(l)": "ранок",
    "mother(r)": "матір",
    "mother(l)": "матір",
    "my(r)": "мій",
    "my(l)": "мій",
    "night(r)": "ніч",
    "night(l)": "ніч",
    "only(r)": "тільки",
    "only(l)": "тільки",
    "salt(r)": "сіль",
    "salt(l)": "сіль",
    "skydiver(r)": "парашутист",
    "skydiver(l)": "парашутист",
    "spoon(r)": "ложка",
    "spoon(l)": "ложка",
    "sun(r)": "сонце",
    "sun(l)": "сонце",
    "team(r)": "команда",
    "team(l)": "команда",
    "thanks(r)": "дякую",
    "thanks(l)": "дякую",
    "tiger": "тигр",
    "tree(r)": "дерево",
    "tree(l)": "дерево",
    "voice(r)": "голос",
    "voice(l)": "голос",
}

## 2.2 Запис відеоданих для навчання

In [3]:
def create_video(name, no_sequences=2, sequence_length=num_of_frames+1, root='videos'):
    # Create the directory for the video if it doesn't exist
    dir_path = os.path.join(root, name)
    os.makedirs(dir_path, exist_ok=True)

    def record_sequence(sequence):
        # Open the camera and set the resolution
        cap = cv2.VideoCapture(1)
        cap.set(3, frame_width)
        cap.set(4, frame_height)

        # Create a video writer to save the frames
        writer = cv2.VideoWriter(
            '{}/{}_{}.mp4'.format(dir_path, name, str(len(os.listdir(dir_path))).zfill(2)),
            cv2.VideoWriter_fourcc(*'DIVX'), frame_rate, (frame_width, frame_height))

        # Loop through the frames
        for j in range(sequence_length):
            ret, frame = cap.read()

            # Show a "Get Ready" message before the first frame
            if j == 0:
                blank_image = cv2.imread('blank.jpg')
                cv2.putText(blank_image, 'GET READY for RECORDING collection {}'.format(name), (10, int(frame_height/2)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(blank_image, 'Collecting frames for {} Video {}'.format(name, sequence), (int(frame_width/2 - 80), 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                cv2.imshow('frame', blank_image)
                cv2.waitKey(2000)
            else:
                # Write the frame to the video writer
                writer.write(cv2.resize(frame, (frame_width, frame_height)))
                cv2.imshow('frame', frame)

            # Quit if the user presses "q"
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        # Release the video writer, camera, and close the window
        writer.release()
        cap.release()
        cv2.destroyAllWindows()

    # Loop through and record multiple sequences
    for n in range(no_sequences):
        record_sequence(n)

# 3. Попередня обробка набору данних (вилучення опорних точок з відео)

## 3.1 Допоміжні функції

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


def mediapipe_detection(image, model):
    # Convert image color space from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    # Set the image as non-writeable
    image.flags.writeable = False                  
    # Use the specified model to make predictions on the image
    results = model.process(image)                 
    # Set the image as writeable again
    image.flags.writeable = True                   
    # Convert the image color space back from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Return the image and the results of the model prediction
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
    
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    # extract pose landmarks and flatten the array
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # extract face landmarks and flatten the array
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    # extract left hand landmarks and flatten the array
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    # extract right hand landmarks and flatten the array
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # concatenate all the landmark arrays into a single feature vector
    return np.concatenate([pose, face, lh, rh])
  

## 3.2 Основна частина

In [5]:
# parse video and create pictures for each frame with landmarks on it
def parse_all_frames(video_basename, basename, number):

    # Get the path of the video to parse
    video_path = 'videos/{}/{}.mp4'.format(basename, video_basename)

    # Set the directory path to save the keypoints
    dir_path = 'keypoints/{}/{}_{}'.format(basename, basename, str(number).zfill(2))
    print('in parse:', basename, number, video_path, dir_path)

    try:
        # Create a directory to save the keypoints if it doesn't exist
        os.makedirs(os.path.join(dir_path))
    except:
        # If the directory already exists, return
        return

    # Open the video for reading
    cap = cv2.VideoCapture(video_path)

    # If the video cannot be opened, return
    if not cap.isOpened():
        return

    # Set the base path for saving keypoints
    base_path = os.path.join(dir_path, basename)

    # Get the number of digits for file names
    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    # Set the initial frame number to 0
    n = 0

    # Create a holistic object for detecting keypoints
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            # Read a frame from the video
            ret, frame = cap.read()

            if ret:
                # Make detections on the frame
                image, results = mediapipe_detection(frame, holistic)

                # Draw the detected keypoints on the frame
                #draw_styled_landmarks(image, results) похоже это не надо

                # Save the extracted keypoints
                np.save('{}_{}'.format(base_path, str(n).zfill(digit)), extract_keypoints(results))

                # Increment the frame number
                n += 1
            else:
                # If there are no more frames to read, return
                return
            
# call parse_all_frames to extract and save keypoints for each frame in video         
def parse_all_videos_for(name):
    for name in os.listdir('videos/{}'.format(name)):
        # extract action name and number from filename
        action_name = name.split('.')[0]
        base_action_name, action_number = action_name.split('_')
        parse_all_frames(action_name, base_action_name, action_number)
        
for name in os.listdir('videos'):
    parse_all_videos_for(name)

in parse: apple(l) 00 videos/apple(l)/apple(l)_00.mp4 keypoints/apple(l)/apple(l)_00
in parse: apple(l) 01 videos/apple(l)/apple(l)_01.mp4 keypoints/apple(l)/apple(l)_01
in parse: apple(l) 02 videos/apple(l)/apple(l)_02.mp4 keypoints/apple(l)/apple(l)_02
in parse: apple(l) 03 videos/apple(l)/apple(l)_03.mp4 keypoints/apple(l)/apple(l)_03
in parse: apple(l) 04 videos/apple(l)/apple(l)_04.mp4 keypoints/apple(l)/apple(l)_04
in parse: apple(l) 05 videos/apple(l)/apple(l)_05.mp4 keypoints/apple(l)/apple(l)_05
in parse: apple(l) 06 videos/apple(l)/apple(l)_06.mp4 keypoints/apple(l)/apple(l)_06
in parse: apple(l) 07 videos/apple(l)/apple(l)_07.mp4 keypoints/apple(l)/apple(l)_07
in parse: apple(l) 08 videos/apple(l)/apple(l)_08.mp4 keypoints/apple(l)/apple(l)_08
in parse: apple(l) 09 videos/apple(l)/apple(l)_09.mp4 keypoints/apple(l)/apple(l)_09
in parse: apple(l) 10 videos/apple(l)/apple(l)_10.mp4 keypoints/apple(l)/apple(l)_10
in parse: apple(l) 11 videos/apple(l)/apple(l)_11.mp4 keypoints/a

## 3.3 Підготовка даних для навчання

In [6]:
#data colection
sequences, labels = [], []
for sign_name in label_map.keys():
    #     path of top folder (keypoints/{name})
    dir_path = os.path.join('keypoints', sign_name)
    for sequence in os.listdir(dir_path):
        #         path to pose folder (keypoints/{name}/{sequence})
        sequence_path = os.path.join(dir_path, sequence)
        window = []
        for id, frame in enumerate(os.listdir(sequence_path)):
            frame_path = os.path.join(sequence_path, frame)
            frame_data = np.load(frame_path)
            window.append(frame_data)
        if len(window) < num_of_frames:
            for i in range(num_of_frames - len(window)):
                window.append(np.array([0 for j in range(1662)]))
        sequences.append(window)
        labels.append(label_map[sign_name])

## 3.4 Розбиття на навчальні та тестові дані

In [7]:
X = np.array(sequences)
print(X.shape)

y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1984, 90, 1662)
(1884, 90, 1662) (100, 90, 1662) (1884, 66) (100, 66)


# 4. Розробка моделей нейроної мережі

## 4.0 Підготовка

In [15]:
tf.config.set_visible_devices([], 'GPU')

log_dir = os.path.join('Logs', 'fit', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb_callback = TensorBoard(log_dir=log_dir)

def print_summary_for(model):
    model.summary()

    yhat = model.predict(X_test)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    print(multilabel_confusion_matrix(ytrue, yhat))

    print(accuracy_score(ytrue, yhat))

## 4.1 MODEL 1

In [10]:
model_1 = Sequential()
model_1.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(90,1662)))
model_1.add(LSTM(96, activation='tanh', return_sequences=True))
model_1.add(LSTM(128, activation='tanh', return_sequences=False))
model_1.add(Dense(192, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(np.array(list(label_map.keys())).shape[0], activation='softmax'))

optimizer=Adam(learning_rate=0.0008)

model_1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_1.fit(X_train, y_train, epochs=150, callbacks=[tb_callback])

Epoch 1/150
59/59 [==============================] - 24s 311ms/step - loss: 4.1943 - categorical_accuracy: 0.0175
Epoch 2/150
59/59 [==============================] - 19s 315ms/step - loss: 4.1432 - categorical_accuracy: 0.0218
Epoch 3/150
59/59 [==============================] - 18s 305ms/step - loss: 3.8622 - categorical_accuracy: 0.0313
Epoch 4/150
59/59 [==============================] - 18s 308ms/step - loss: 3.7178 - categorical_accuracy: 0.0441
Epoch 5/150
59/59 [==============================] - 19s 316ms/step - loss: 3.5838 - categorical_accuracy: 0.0504
Epoch 6/150
59/59 [==============================] - 19s 317ms/step - loss: 3.4643 - categorical_accuracy: 0.0547
Epoch 7/150
59/59 [==============================] - 19s 320ms/step - loss: 3.4516 - categorical_accuracy: 0.0552
Epoch 8/150
59/59 [==============================] - 19s 319ms/step - loss: 3.3086 - categorical_accuracy: 0.0711
Epoch 9/150
59/59 [==============================] - 19s 326ms/step - loss: 3.4590 - cat

59/59 [==============================] - 20s 334ms/step - loss: 0.4947 - categorical_accuracy: 0.8179
Epoch 73/150
59/59 [==============================] - 20s 332ms/step - loss: 0.4678 - categorical_accuracy: 0.8286
Epoch 74/150
59/59 [==============================] - 20s 342ms/step - loss: 0.5589 - categorical_accuracy: 0.7967
Epoch 75/150
59/59 [==============================] - 21s 353ms/step - loss: 0.6975 - categorical_accuracy: 0.7590
Epoch 76/150
59/59 [==============================] - 20s 340ms/step - loss: 0.5641 - categorical_accuracy: 0.8025
Epoch 77/150
59/59 [==============================] - 20s 347ms/step - loss: 0.5900 - categorical_accuracy: 0.7898
Epoch 78/150
59/59 [==============================] - 21s 349ms/step - loss: 0.4460 - categorical_accuracy: 0.8408
Epoch 79/150
59/59 [==============================] - 20s 344ms/step - loss: 0.3990 - categorical_accuracy: 0.8599
Epoch 80/150
59/59 [==============================] - 20s 333ms/step - loss: 0.7997 - categor

KeyboardInterrupt: 

In [16]:
print_summary_for(model_1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 90, 128)           916992    
                                                                 
 lstm_1 (LSTM)               (None, 90, 96)            86400     
                                                                 
 lstm_2 (LSTM)               (None, 128)               115200    
                                                                 
 dense (Dense)               (None, 192)               24768     
                                                                 
 dropout (Dropout)           (None, 192)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                12352     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

## 4.2 MODEL 2

In [18]:
model_2 = Sequential()
model_2.add(LSTM(256, return_sequences=True, activation='tanh', input_shape=(90, 1662)))
model_2.add(Dropout(0.2))
model_2.add(LSTM(64, return_sequences=True, activation='tanh'))
model_2.add(Dropout(0.2))
model_2.add(LSTM(64, return_sequences=False, activation='tanh'))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(np.array(list(label_map.keys())).shape[0], activation='softmax'))

model_2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_2.fit(X_train, y_train, epochs=170, callbacks=[tb_callback])

Epoch 1/170
59/59 [==============================] - 50s 768ms/step - loss: 4.1990 - categorical_accuracy: 0.0133
Epoch 2/170
59/59 [==============================] - 50s 844ms/step - loss: 4.1923 - categorical_accuracy: 0.0127
Epoch 3/170
59/59 [==============================] - 52s 875ms/step - loss: 4.1920 - categorical_accuracy: 0.0106
Epoch 4/170
59/59 [==============================] - 55s 924ms/step - loss: 4.1916 - categorical_accuracy: 0.0175
Epoch 5/170
59/59 [==============================] - 52s 878ms/step - loss: 4.1914 - categorical_accuracy: 0.0111
Epoch 6/170
59/59 [==============================] - 53s 904ms/step - loss: 4.1907 - categorical_accuracy: 0.0159
Epoch 7/170
59/59 [==============================] - 54s 912ms/step - loss: 4.1905 - categorical_accuracy: 0.0111
Epoch 8/170
59/59 [==============================] - 51s 867ms/step - loss: 4.1904 - categorical_accuracy: 0.0186
Epoch 9/170
59/59 [==============================] - 53s 897ms/step - loss: 4.0879 - cat

59/59 [==============================] - 47s 797ms/step - loss: 0.9827 - categorical_accuracy: 0.6300
Epoch 73/170
59/59 [==============================] - 45s 767ms/step - loss: 0.9621 - categorical_accuracy: 0.6343
Epoch 74/170
59/59 [==============================] - 46s 783ms/step - loss: 0.9931 - categorical_accuracy: 0.6332
Epoch 75/170
59/59 [==============================] - 54s 920ms/step - loss: 0.9424 - categorical_accuracy: 0.6492
Epoch 76/170
59/59 [==============================] - 51s 870ms/step - loss: 1.0522 - categorical_accuracy: 0.6077
Epoch 77/170
59/59 [==============================] - 59s 996ms/step - loss: 0.9242 - categorical_accuracy: 0.6608
Epoch 78/170
59/59 [==============================] - 57s 965ms/step - loss: 0.8503 - categorical_accuracy: 0.6757
Epoch 79/170
59/59 [==============================] - 53s 896ms/step - loss: 0.8377 - categorical_accuracy: 0.6805
Epoch 80/170
59/59 [==============================] - 48s 807ms/step - loss: 0.8604 - categor

Epoch 143/170
59/59 [==============================] - 53s 896ms/step - loss: 0.3970 - categorical_accuracy: 0.8450
Epoch 144/170
59/59 [==============================] - 52s 875ms/step - loss: 0.3218 - categorical_accuracy: 0.8758
Epoch 145/170
59/59 [==============================] - 56s 946ms/step - loss: 0.3309 - categorical_accuracy: 0.8678
Epoch 146/170
59/59 [==============================] - 48s 805ms/step - loss: 0.3236 - categorical_accuracy: 0.8700
Epoch 147/170
59/59 [==============================] - 47s 797ms/step - loss: 0.3165 - categorical_accuracy: 0.8779
Epoch 148/170
59/59 [==============================] - 47s 798ms/step - loss: 0.3577 - categorical_accuracy: 0.8572
Epoch 149/170
59/59 [==============================] - 47s 794ms/step - loss: 0.3441 - categorical_accuracy: 0.8668
Epoch 150/170
59/59 [==============================] - 48s 806ms/step - loss: 0.3752 - categorical_accuracy: 0.8620
Epoch 151/170
59/59 [==============================] - 46s 785ms/step - 

In [19]:
print_summary_for(model_2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 90, 256)           1965056   
                                                                 
 dropout_9 (Dropout)         (None, 90, 256)           0         
                                                                 
 lstm_12 (LSTM)              (None, 90, 64)            82176     
                                                                 
 dropout_10 (Dropout)        (None, 90, 64)            0         
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dense_11 (Dense)            (None, 256)               16640     
                                                                 
 dropout_11 (Dropout)        (None, 256)              

## 4.3 MODEL 3

In [14]:
model_3 = Sequential()
model_3.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(90,1662)))
model_3.add(LSTM(128, activation='tanh', return_sequences=False))
model_3.add(Dense(192, activation='relu'))
model_3.add(Dropout(0.2))
model_3.add(Dense(np.array(list(label_map.keys())).shape[0], activation='softmax'))

model_3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_3.fit(X_train, y_train, epochs=80, callbacks=[tb_callback])

Epoch 1/80
59/59 [==============================] - 23s 332ms/step - loss: 4.2201 - categorical_accuracy: 0.0122
Epoch 2/80
59/59 [==============================] - 19s 323ms/step - loss: 4.1914 - categorical_accuracy: 0.0218
Epoch 3/80
59/59 [==============================] - 19s 324ms/step - loss: 3.9011 - categorical_accuracy: 0.0297
Epoch 4/80
59/59 [==============================] - 19s 328ms/step - loss: 3.4947 - categorical_accuracy: 0.0515
Epoch 5/80
59/59 [==============================] - 20s 334ms/step - loss: 3.3777 - categorical_accuracy: 0.0637
Epoch 6/80
59/59 [==============================] - 23s 387ms/step - loss: 3.3206 - categorical_accuracy: 0.0801
Epoch 7/80
59/59 [==============================] - 20s 335ms/step - loss: 3.1256 - categorical_accuracy: 0.1184
Epoch 8/80
59/59 [==============================] - 20s 335ms/step - loss: 2.9270 - categorical_accuracy: 0.1561
Epoch 9/80
59/59 [==============================] - 20s 341ms/step - loss: 2.9393 - categorical_

KeyboardInterrupt: 

In [17]:
print_summary_for(model_3)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 90, 128)           916992    
                                                                 
 lstm_10 (LSTM)              (None, 128)               131584    
                                                                 
 dense_9 (Dense)             (None, 192)               24768     
                                                                 
 dropout_8 (Dropout)         (None, 192)               0         
                                                                 
 dense_10 (Dense)            (None, 66)                12738     
                                                                 
Total params: 1,086,082
Trainable params: 1,086,082
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 1s 89

## 4.4 MODEL 4

In [20]:
model_4 = Sequential()
model_4.add(LSTM(192, return_sequences=True, activation='sigmoid', input_shape=(90,1662)))
model_4.add(LSTM(192, activation='sigmoid', return_sequences=False))
model_4.add(Dense(128, activation='selu'))
model_4.add(Dropout(0.2))
model_4.add(Dense(np.array(list(label_map.keys())).shape[0], activation='softmax'))

model_4.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_4.fit(X_train, y_train, epochs=81, callbacks=[tb_callback])

Epoch 1/81
59/59 [==============================] - 37s 600ms/step - loss: 4.2931 - categorical_accuracy: 0.0106
Epoch 2/81
59/59 [==============================] - 37s 627ms/step - loss: 4.2194 - categorical_accuracy: 0.0106
Epoch 3/81
59/59 [==============================] - 38s 649ms/step - loss: 4.1098 - categorical_accuracy: 0.0244
Epoch 4/81
59/59 [==============================] - 38s 647ms/step - loss: 3.9219 - categorical_accuracy: 0.0329
Epoch 5/81
59/59 [==============================] - 39s 657ms/step - loss: 3.6781 - categorical_accuracy: 0.0499
Epoch 6/81
59/59 [==============================] - 40s 683ms/step - loss: 3.4057 - categorical_accuracy: 0.0727
Epoch 7/81
59/59 [==============================] - 41s 687ms/step - loss: 3.1717 - categorical_accuracy: 0.0886
Epoch 8/81
59/59 [==============================] - 42s 710ms/step - loss: 3.0084 - categorical_accuracy: 0.0945
Epoch 9/81
59/59 [==============================] - 44s 741ms/step - loss: 2.8844 - categorical_

Epoch 73/81
59/59 [==============================] - 41s 692ms/step - loss: 0.2591 - categorical_accuracy: 0.9087
Epoch 74/81
59/59 [==============================] - 45s 760ms/step - loss: 0.2745 - categorical_accuracy: 0.9034
Epoch 75/81
59/59 [==============================] - 43s 721ms/step - loss: 0.2742 - categorical_accuracy: 0.9002
Epoch 76/81
59/59 [==============================] - 44s 738ms/step - loss: 0.2823 - categorical_accuracy: 0.8986
Epoch 77/81
59/59 [==============================] - 43s 726ms/step - loss: 0.2775 - categorical_accuracy: 0.9039
Epoch 78/81
59/59 [==============================] - 47s 799ms/step - loss: 0.2735 - categorical_accuracy: 0.8992
Epoch 79/81
59/59 [==============================] - 44s 737ms/step - loss: 0.3938 - categorical_accuracy: 0.8646
Epoch 80/81
59/59 [==============================] - 42s 704ms/step - loss: 0.3352 - categorical_accuracy: 0.8859
Epoch 81/81
59/59 [==============================] - 41s 689ms/step - loss: 0.2405 - cat

In [21]:
print_summary_for(model_4)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 90, 192)           1424640   
                                                                 
 lstm_15 (LSTM)              (None, 192)               295680    
                                                                 
 dense_14 (Dense)            (None, 128)               24704     
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 66)                8514      
                                                                 
Total params: 1,753,538
Trainable params: 1,753,538
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 1s 15

## 4.5 MODEL 5

In [34]:
model_5 = Sequential()
model_5.add(LSTM(128, return_sequences=True, activation='sigmoid', input_shape=(90,1662)))
model_5.add(LSTM(128, activation='sigmoid', return_sequences=False))
model_5.add(Dense(192, activation='selu'))
model_5.add(Dropout(0.2))
model_5.add(Dense(np.array(list(label_map.keys())).shape[0], activation='softmax'))

model_5.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_5.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
59/59 [==============================] - 19s 278ms/step - loss: 4.2649 - categorical_accuracy: 0.0106
Epoch 2/100
59/59 [==============================] - 17s 280ms/step - loss: 4.2206 - categorical_accuracy: 0.0122
Epoch 3/100
59/59 [==============================] - 17s 282ms/step - loss: 4.1693 - categorical_accuracy: 0.0234
Epoch 4/100
59/59 [==============================] - 17s 284ms/step - loss: 3.9718 - categorical_accuracy: 0.0329
Epoch 5/100
59/59 [==============================] - 17s 281ms/step - loss: 3.6906 - categorical_accuracy: 0.0425
Epoch 6/100
59/59 [==============================] - 17s 289ms/step - loss: 3.4904 - categorical_accuracy: 0.0674
Epoch 7/100
59/59 [==============================] - 17s 287ms/step - loss: 3.3363 - categorical_accuracy: 0.0727
Epoch 8/100
59/59 [==============================] - 17s 289ms/step - loss: 3.0901 - categorical_accuracy: 0.1109
Epoch 9/100
59/59 [==============================] - 17s 284ms/step - loss: 2.8888 - cat

59/59 [==============================] - 18s 307ms/step - loss: 0.3634 - categorical_accuracy: 0.8631
Epoch 73/100
59/59 [==============================] - 19s 321ms/step - loss: 0.3216 - categorical_accuracy: 0.8875
Epoch 74/100
59/59 [==============================] - 19s 330ms/step - loss: 0.3336 - categorical_accuracy: 0.8816
Epoch 75/100
59/59 [==============================] - 19s 329ms/step - loss: 0.3527 - categorical_accuracy: 0.8726
Epoch 76/100
59/59 [==============================] - 19s 327ms/step - loss: 0.3187 - categorical_accuracy: 0.8843
Epoch 77/100
59/59 [==============================] - 18s 310ms/step - loss: 0.3118 - categorical_accuracy: 0.8885
Epoch 78/100
59/59 [==============================] - 19s 319ms/step - loss: 0.2743 - categorical_accuracy: 0.9087
Epoch 79/100
59/59 [==============================] - 18s 312ms/step - loss: 0.2770 - categorical_accuracy: 0.9029
Epoch 80/100
59/59 [==============================] - 19s 316ms/step - loss: 0.2665 - categor

In [35]:
print_summary_for(model_5)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 90, 128)           916992    
                                                                 
 lstm_23 (LSTM)              (None, 128)               131584    
                                                                 
 dense_22 (Dense)            (None, 192)               24768     
                                                                 
 dropout_16 (Dropout)        (None, 192)               0         
                                                                 
 dense_23 (Dense)            (None, 66)                12738     
                                                                 
Total params: 1,086,082
Trainable params: 1,086,082
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 1s 11

# 5. Тестування розроблених моделей

## 5.0 Допоміжні функції

In [24]:
def secondmax(arr, maxind):
    sublist = [x for x in arr]
    sublist[maxind] = 0
    return np.argmax(sublist)

def sign_from_video(path, model=model_5):
    # Open video file
    cap = cv2.VideoCapture(path)

    # Check if video file was successfully opened
    if not cap.isOpened():
        return 'unknow (1)'  # Return string indicating video file was not successfully opened

    # Initialize empty list to store keypoints
    keypoints = []
    
    # Initialize Holistic model for full-body pose detection and tracking
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            # Read a frame from the video file
            ret, frame = cap.read()

            if ret:
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                        
                # Extract keypoints from the results of the pose detection
                keypoints.append(extract_keypoints(results))
            else:
                # No more frames to read
                break
    
    # Use the trained model to predict the sign language gesture from the extracted keypoints
    res = model.predict(np.expand_dims(keypoints, axis=0))[0]
    first = np.argmax(res)
    second = secondmax(res, first)

    # Release the video file
    cap.release()

    # Return a string indicating the predicted sign language gesture,
    # as well as the top two predicted gestures and their corresponding probabilities
    return ('{}({:.0f}%); {}({:.0f}%)'.format(actions[first], res[first] * 100, actions[second], res[second] * 100) 
            or 'unknow (2)', actions[first], actions[second])

#A function for testing the trained model on test data.
def test(model, root='TEST_DATA'):
    
    res_dict = {} # initialize an empty dictionary to store the results
    for fold in os.listdir(root): # iterate over the test data folders
        print("Going through folder ", fold)
        for vid in os.listdir(os.path.join(root, fold)): # iterate over the videos in each folder
            string, first, second = sign_from_video(os.path.join(root, fold, vid), model=model) # use the trained model to predict the sign from the video
            first = first.split('(')[0] # extract the predicted sign from the string
            if fold == first: # if the predicted sign matches the folder name, consider the prediction correct
                try:
                    if res_dict[first] > 0: # if the predicted sign is already in the dictionary, increment the count
                        res_dict[first] = res_dict[first] + 1
                except: # if the predicted sign is not in the dictionary, add it
                    res_dict[first] = 1                    
            print('    ', string) # print the prediction
    print('Result:')
    print(res_dict) # print the dictionary with the results

## 5.1 MODEL 1

In [25]:
model_1.load_weights('MODEL1.h5')
test(model_1)

Going through folder  apple
1/1 [==============================] - 0s 59ms/step
     apple(l)(96%); goodbye(l)(2%)
1/1 [==============================] - 0s 47ms/step
     apple(l)(71%); goodbye(l)(20%)
1/1 [==============================] - 0s 35ms/step
     apple(r)(60%); goodbye(r)(37%)
1/1 [==============================] - 0s 31ms/step
     apple(l)(47%); fruit(l)(42%)
1/1 [==============================] - 0s 32ms/step
     apple(l)(96%); goodbye(l)(2%)
1/1 [==============================] - 0s 31ms/step
     apple(r)(91%); fruit(r)(7%)
1/1 [==============================] - 0s 32ms/step
     apple(r)(70%); hello(r)(13%)
1/1 [==============================] - 0s 32ms/step
     apple(r)(67%); goodbye(r)(19%)
1/1 [==============================] - 0s 33ms/step
     apple(r)(95%); fruit(r)(4%)
1/1 [==============================] - 0s 30ms/step
     goodbye(r)(54%); apple(r)(44%)
Going through folder  ball
1/1 [==============================] - 0s 31ms/step
     ball(66%); brother(l

1/1 [==============================] - 0s 39ms/step
     team(r)(49%); interpreter(22%)
1/1 [==============================] - 0s 31ms/step
     guitar(l)(48%); interpreter(20%)
1/1 [==============================] - 0s 30ms/step
     interpreter(76%); guitar(r)(9%)
1/1 [==============================] - 0s 30ms/step
     butterfly(79%); guitar(l)(9%)
Going through folder  day
1/1 [==============================] - 0s 30ms/step
     day(100%); brother(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 31ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 45ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 34ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 31ms/step
     day(100%); team(r)(0%)
1/1 [==============================] - 0s 31ms/step
     

1/1 [==============================] - 0s 33ms/step
     goodbye(l)(90%); apple(l)(6%)
1/1 [==============================] - 0s 30ms/step
     hello(r)(58%); fruit(r)(27%)
Going through folder  home
1/1 [==============================] - 0s 32ms/step
     home(85%); car(l)(13%)
1/1 [==============================] - 0s 33ms/step
     home(100%); book(0%)
1/1 [==============================] - 0s 30ms/step
     home(64%); book(24%)
1/1 [==============================] - 0s 31ms/step
     tiger(73%); home(18%)
1/1 [==============================] - 0s 31ms/step
     tiger(55%); home(27%)
1/1 [==============================] - 0s 30ms/step
     book(50%); home(38%)
1/1 [==============================] - 0s 31ms/step
     tiger(67%); home(22%)
1/1 [==============================] - 0s 33ms/step
     home(56%); book(22%)
1/1 [==============================] - 0s 32ms/step
     home(44%); book(38%)
1/1 [==============================] - 0s 31ms/step
     home(63%); book(35%)
Going through f

1/1 [==============================] - 0s 29ms/step
     only(l)(99%); bike(0%)
1/1 [==============================] - 0s 31ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 35ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 31ms/step
     only(r)(100%); skydiver(l)(0%)
1/1 [==============================] - 0s 34ms/step
     only(r)(59%); chess(41%)
1/1 [==============================] - 0s 31ms/step
     only(r)(90%); chess(10%)
1/1 [==============================] - 0s 33ms/step
     only(l)(100%); bowl(0%)
Going through folder  salt
1/1 [==============================] - 0s 32ms/step
     hello(r)(58%); salt(r)(40%)
1/1 [==============================] - 0s 30ms/step
     salt(r)(58%); fruit(r)(32%)
1/1 [==============================] - 0s 35ms/step
     spoon(r)(59%); salt(r)(21%)
1/1 [==============================] - 0s 31ms/step
     spoon(r)(47%); salt(r)(26%)
1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 33ms/step
     voice(l)(55%); friend(l)(34%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(100%); friend(l)(0%)
1/1 [==============================] - 0s 32ms/step
     voice(r)(67%); morning(r)(32%)
Result:
{'apple': 9, 'ball': 7, 'bike': 9, 'book': 8, 'bowl': 6, 'boy': 7, 'brother': 7, 'butterfly': 3, 'car': 8, 'chess': 3, 'day': 10, 'evening': 8, 'father': 5, 'friend': 9, 'fruit': 9, 'good': 7, 'goodbye': 8, 'guitar': 9, 'hello': 3, 'home': 6, 'I': 10, 'interpreter': 3, 'island': 9, 'medal': 8, 'morning': 10, 'mother': 9, 'my': 1, 'night': 8, 'only': 10, 'salt': 3, 'skydiver': 9, 'spoon': 9, 'sun': 9, 'team': 7, 'thanks': 6, 'tiger': 9, 'tree': 9, 'voice': 9}


## 5.2 MODEL 2

In [26]:
model_2.load_weights('MODEL2.h5')
test(model_2)

Going through folder  apple
1/1 [==============================] - 0s 42ms/step
     apple(l)(100%); thanks(l)(0%)
1/1 [==============================] - 0s 38ms/step
     apple(l)(99%); goodbye(l)(0%)
1/1 [==============================] - 0s 35ms/step
     apple(r)(85%); goodbye(r)(13%)
1/1 [==============================] - 0s 36ms/step
     apple(l)(90%); fruit(l)(8%)
1/1 [==============================] - 0s 34ms/step
     apple(l)(99%); goodbye(l)(0%)
1/1 [==============================] - 0s 42ms/step
     sun(r)(73%); apple(r)(14%)
1/1 [==============================] - 0s 35ms/step
     apple(r)(84%); fruit(r)(8%)
1/1 [==============================] - 0s 34ms/step
     apple(r)(77%); fruit(r)(15%)
1/1 [==============================] - 0s 35ms/step
     apple(r)(92%); goodbye(r)(5%)
1/1 [==============================] - 0s 34ms/step
     goodbye(r)(41%); apple(r)(39%)
Going through folder  ball
1/1 [==============================] - 0s 35ms/step
     ball(100%); bowl(0%)
1/1

     chess(51%); guitar(r)(33%)
1/1 [==============================] - 0s 41ms/step
     team(l)(79%); interpreter(13%)
1/1 [==============================] - 0s 42ms/step
     guitar(r)(97%); chess(2%)
1/1 [==============================] - 0s 40ms/step
     team(l)(73%); guitar(l)(13%)
1/1 [==============================] - 0s 42ms/step
     guitar(r)(49%); guitar(l)(33%)
Going through folder  day
1/1 [==============================] - 0s 42ms/step
     day(100%); skydiver(r)(0%)
1/1 [==============================] - 0s 38ms/step
     day(100%); skydiver(r)(0%)
1/1 [==============================] - 0s 44ms/step
     day(100%); chess(0%)
1/1 [==============================] - 0s 40ms/step
     day(100%); ball(0%)
1/1 [==============================] - 0s 44ms/step
     day(100%); ball(0%)
1/1 [==============================] - 0s 43ms/step
     day(100%); only(r)(0%)
1/1 [==============================] - 0s 40ms/step
     day(100%); ball(0%)
1/1 [==============================] - 0

1/1 [==============================] - 0s 39ms/step
     fruit(r)(44%); hello(r)(35%)
Going through folder  home
1/1 [==============================] - 0s 38ms/step
     friend(l)(78%); voice(l)(21%)
1/1 [==============================] - 0s 42ms/step
     home(85%); friend(l)(15%)
1/1 [==============================] - 0s 43ms/step
     home(65%); book(24%)
1/1 [==============================] - 0s 41ms/step
     home(84%); book(15%)
1/1 [==============================] - 0s 61ms/step
     home(57%); book(42%)
1/1 [==============================] - 0s 41ms/step
     book(58%); home(38%)
1/1 [==============================] - 0s 40ms/step
     book(55%); home(44%)
1/1 [==============================] - 0s 38ms/step
     book(64%); home(32%)
1/1 [==============================] - 0s 39ms/step
     book(54%); home(46%)
1/1 [==============================] - 0s 39ms/step
     book(80%); home(19%)
Going through folder  I
1/1 [==============================] - 0s 40ms/step
     medal(r)(42%

1/1 [==============================] - 0s 40ms/step
     only(l)(100%); interpreter(0%)
1/1 [==============================] - 0s 45ms/step
     only(l)(100%); team(r)(0%)
1/1 [==============================] - 0s 41ms/step
     only(r)(99%); interpreter(0%)
1/1 [==============================] - 0s 41ms/step
     chess(76%); only(r)(23%)
1/1 [==============================] - 0s 37ms/step
     only(r)(58%); chess(24%)
1/1 [==============================] - 0s 38ms/step
     only(l)(98%); bowl(2%)
Going through folder  salt
1/1 [==============================] - 0s 42ms/step
     salt(r)(49%); hello(r)(44%)
1/1 [==============================] - 0s 41ms/step
     salt(r)(51%); fruit(r)(35%)
1/1 [==============================] - 0s 49ms/step
     hello(r)(56%); salt(r)(35%)
1/1 [==============================] - 0s 43ms/step
     hello(r)(47%); salt(r)(29%)
1/1 [==============================] - 0s 37ms/step
     salt(l)(86%); hello(l)(13%)
1/1 [==============================] - 0s 39m

1/1 [==============================] - 0s 42ms/step
     voice(l)(99%); friend(r)(1%)
1/1 [==============================] - 0s 38ms/step
     morning(r)(99%); friend(r)(1%)
Result:
{'apple': 8, 'ball': 7, 'bike': 8, 'book': 9, 'bowl': 7, 'boy': 7, 'brother': 7, 'butterfly': 6, 'car': 10, 'chess': 4, 'day': 10, 'evening': 10, 'father': 6, 'friend': 9, 'fruit': 9, 'good': 9, 'goodbye': 4, 'guitar': 10, 'hello': 6, 'home': 4, 'I': 7, 'interpreter': 4, 'island': 9, 'medal': 4, 'morning': 10, 'mother': 7, 'my': 4, 'night': 8, 'only': 8, 'salt': 5, 'skydiver': 8, 'spoon': 8, 'sun': 9, 'team': 7, 'thanks': 6, 'tiger': 9, 'tree': 7, 'voice': 8}


## 5.3 MODEL 3

In [27]:
model_3.load_weights('MODEL3.h5')
test(model_3)

Going through folder  apple
1/1 [==============================] - 0s 49ms/step
     apple(l)(88%); fruit(l)(11%)
1/1 [==============================] - 0s 29ms/step
     apple(l)(79%); fruit(l)(19%)
1/1 [==============================] - 0s 32ms/step
     apple(r)(92%); fruit(r)(4%)
1/1 [==============================] - 0s 33ms/step
     apple(l)(63%); fruit(l)(35%)
1/1 [==============================] - 0s 30ms/step
     apple(l)(85%); fruit(l)(14%)
1/1 [==============================] - 0s 28ms/step
     apple(r)(87%); hello(r)(7%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(76%); hello(r)(14%)
1/1 [==============================] - 0s 33ms/step
     apple(r)(66%); hello(r)(18%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(95%); fruit(r)(2%)
1/1 [==============================] - 0s 31ms/step
     goodbye(r)(67%); hello(r)(13%)
Going through folder  ball
1/1 [==============================] - 0s 29ms/step
     ball(99%); brother(l)(0%)
1/1

1/1 [==============================] - 0s 28ms/step
     chess(62%); interpreter(26%)
1/1 [==============================] - 0s 28ms/step
     chess(93%); good(r)(3%)
1/1 [==============================] - 0s 28ms/step
     good(r)(63%); chess(30%)
1/1 [==============================] - 0s 27ms/step
     chess(55%); car(l)(43%)
Going through folder  day
1/1 [==============================] - 0s 33ms/step
     day(100%); car(r)(0%)
1/1 [==============================] - 0s 26ms/step
     day(99%); car(r)(1%)
1/1 [==============================] - 0s 29ms/step
     day(97%); book(3%)
1/1 [==============================] - 0s 26ms/step
     day(100%); book(0%)
1/1 [==============================] - 0s 26ms/step
     day(100%); book(0%)
1/1 [==============================] - 0s 28ms/step
     day(100%); book(0%)
1/1 [==============================] - 0s 26ms/step
     day(100%); book(0%)
1/1 [==============================] - 0s 26ms/step
     day(100%); book(0%)
1/1 [=====================

1/1 [==============================] - 0s 26ms/step
     hello(r)(80%); goodbye(r)(8%)
Going through folder  home
1/1 [==============================] - 0s 28ms/step
     home(81%); book(18%)
1/1 [==============================] - 0s 27ms/step
     home(100%); book(0%)
1/1 [==============================] - 0s 37ms/step
     home(56%); book(43%)
1/1 [==============================] - 0s 27ms/step
     home(97%); book(2%)
1/1 [==============================] - 0s 28ms/step
     home(78%); book(21%)
1/1 [==============================] - 0s 26ms/step
     home(88%); book(8%)
1/1 [==============================] - 0s 27ms/step
     book(55%); home(44%)
1/1 [==============================] - 0s 27ms/step
     home(90%); book(8%)
1/1 [==============================] - 0s 26ms/step
     home(87%); book(12%)
1/1 [==============================] - 0s 26ms/step
     home(89%); book(10%)
Going through folder  I
1/1 [==============================] - 0s 27ms/step
     I(r)(68%); medal(r)(22%)
1/1

1/1 [==============================] - 0s 27ms/step
     only(l)(99%); good(l)(1%)
1/1 [==============================] - 0s 27ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 27ms/step
     only(r)(96%); chess(3%)
1/1 [==============================] - 0s 27ms/step
     chess(90%); bowl(3%)
1/1 [==============================] - 0s 28ms/step
     chess(99%); bike(1%)
1/1 [==============================] - 0s 27ms/step
     only(l)(100%); good(l)(0%)
Going through folder  salt
1/1 [==============================] - 0s 27ms/step
     salt(r)(65%); hello(r)(34%)
1/1 [==============================] - 0s 27ms/step
     salt(r)(97%); hello(r)(2%)
1/1 [==============================] - 0s 27ms/step
     salt(r)(71%); hello(r)(22%)
1/1 [==============================] - 0s 27ms/step
     salt(r)(75%); hello(r)(17%)
1/1 [==============================] - 0s 27ms/step
     salt(l)(59%); hello(l)(20%)
1/1 [==============================] - 0s 27ms/step
     salt

1/1 [==============================] - 0s 27ms/step
     voice(l)(73%); friend(l)(20%)
1/1 [==============================] - 0s 27ms/step
     voice(r)(62%); morning(r)(35%)
Result:
{'apple': 9, 'ball': 6, 'bike': 7, 'book': 10, 'bowl': 9, 'boy': 4, 'brother': 9, 'butterfly': 6, 'car': 8, 'chess': 7, 'day': 10, 'evening': 10, 'father': 5, 'friend': 8, 'fruit': 9, 'good': 9, 'goodbye': 8, 'guitar': 9, 'hello': 7, 'home': 9, 'I': 10, 'interpreter': 5, 'island': 10, 'medal': 7, 'morning': 10, 'mother': 8, 'my': 5, 'night': 10, 'only': 8, 'salt': 7, 'skydiver': 7, 'spoon': 6, 'sun': 6, 'team': 9, 'thanks': 9, 'tiger': 10, 'tree': 10, 'voice': 7}


## 5.4 MODEL 4

In [28]:
model_4.load_weights('MODEL4.h5')
test(model_4)

Going through folder  apple
1/1 [==============================] - 0s 35ms/step
     apple(l)(77%); fruit(l)(21%)
1/1 [==============================] - 0s 31ms/step
     apple(l)(79%); fruit(l)(20%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(96%); fruit(r)(3%)
1/1 [==============================] - 0s 31ms/step
     fruit(l)(87%); apple(l)(11%)
1/1 [==============================] - 0s 31ms/step
     fruit(l)(90%); apple(l)(7%)
1/1 [==============================] - 0s 32ms/step
     apple(r)(99%); mother(l)(0%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(99%); fruit(r)(1%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(98%); fruit(r)(1%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(99%); fruit(r)(1%)
1/1 [==============================] - 0s 32ms/step
     apple(r)(83%); goodbye(r)(10%)
Going through folder  ball
1/1 [==============================] - 0s 32ms/step
     ball(96%); brother(l)(2%)
1/1 [

1/1 [==============================] - 0s 31ms/step
     team(r)(96%); guitar(l)(1%)
1/1 [==============================] - 0s 40ms/step
     team(r)(86%); chess(7%)
1/1 [==============================] - 0s 30ms/step
     team(r)(76%); car(l)(17%)
1/1 [==============================] - 0s 30ms/step
     team(r)(64%); guitar(l)(29%)
1/1 [==============================] - 0s 30ms/step
     car(l)(65%); team(r)(18%)
Going through folder  day
1/1 [==============================] - 0s 30ms/step
     day(100%); skydiver(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(100%); skydiver(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(100%); tree(l)(0%)
1/1 [==============================] - 0s 30ms/step
     day(100%); home(0%)
1/1 [==============================] - 0s 30ms/step
     day(99%); tree(l)(1%)
1/1 [==============================] - 0s 30ms/step
     day(100%); tree(l)(0%)
1/1 [==============================] - 0s 32ms/step
     day(100%); tree

1/1 [==============================] - 0s 30ms/step
     goodbye(l)(67%); fruit(l)(10%)
1/1 [==============================] - 0s 30ms/step
     goodbye(l)(62%); fruit(l)(17%)
1/1 [==============================] - 0s 30ms/step
     hello(r)(95%); goodbye(r)(2%)
Going through folder  home
1/1 [==============================] - 0s 30ms/step
     voice(l)(88%); home(11%)
1/1 [==============================] - 0s 30ms/step
     home(97%); friend(r)(2%)
1/1 [==============================] - 0s 31ms/step
     home(98%); day(1%)
1/1 [==============================] - 0s 30ms/step
     day(83%); home(17%)
1/1 [==============================] - 0s 30ms/step
     home(98%); day(2%)
1/1 [==============================] - 0s 30ms/step
     home(91%); day(9%)
1/1 [==============================] - 0s 30ms/step
     home(97%); day(1%)
1/1 [==============================] - 0s 30ms/step
     home(97%); day(3%)
1/1 [==============================] - 0s 40ms/step
     home(94%); day(5%)
1/1 [========

1/1 [==============================] - 0s 40ms/step
     only(r)(100%); interpreter(0%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); team(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(r)(99%); team(r)(1%)
1/1 [==============================] - 0s 30ms/step
     only(r)(90%); team(r)(7%)
1/1 [==============================] - 0s 30ms/step
     only(r)(92%); team(r)(6%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
Going through folder  salt
1/1 [==============================] - 0s 30ms/step
     salt(r)(81%); hello(r)(17%)
1/1 [==============================] - 0s 30ms/step
     salt(r)(78%); hello(r)(19%)
1/1 [==============================] - 0s 30ms

1/1 [==============================] - 0s 30ms/step
     voice(l)(99%); goodbye(l)(1%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(97%); goodbye(l)(2%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(83%); friend(l)(13%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(54%); friend(l)(34%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(92%); friend(l)(4%)
1/1 [==============================] - 0s 30ms/step
     morning(r)(88%); friend(r)(9%)
Result:
{'apple': 8, 'ball': 10, 'bike': 8, 'book': 10, 'bowl': 8, 'boy': 4, 'brother': 10, 'butterfly': 8, 'car': 10, 'chess': 2, 'day': 10, 'evening': 10, 'father': 5, 'friend': 10, 'fruit': 10, 'good': 10, 'goodbye': 7, 'guitar': 7, 'hello': 6, 'home': 8, 'I': 10, 'interpreter': 2, 'island': 9, 'medal': 6, 'morning': 10, 'mother': 10, 'my': 7, 'night': 10, 'only': 10, 'salt': 8, 'skydiver': 9, 'spoon': 7, 'sun': 10, 'team': 8, 'thanks': 10, 'tiger': 10, 'tree': 10, 'voic

## 5.5 MODEL 5

In [29]:
model_5.load_weights('MODEL5.h5')
test(model_5)

Going through folder  apple
1/1 [==============================] - 0s 30ms/step
     apple(l)(57%); fruit(l)(42%)
1/1 [==============================] - 0s 30ms/step
     apple(l)(66%); fruit(l)(33%)
1/1 [==============================] - 0s 20ms/step
     apple(r)(58%); fruit(r)(42%)
1/1 [==============================] - 0s 30ms/step
     apple(l)(61%); fruit(l)(26%)
1/1 [==============================] - 0s 20ms/step
     fruit(l)(49%); apple(l)(46%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(80%); fruit(r)(19%)
1/1 [==============================] - 0s 30ms/step
     apple(r)(75%); fruit(r)(24%)
1/1 [==============================] - 0s 20ms/step
     apple(r)(70%); fruit(r)(28%)
1/1 [==============================] - 0s 31ms/step
     apple(r)(71%); fruit(r)(29%)
1/1 [==============================] - 0s 20ms/step
     fruit(r)(52%); apple(r)(47%)
Going through folder  ball
1/1 [==============================] - 0s 30ms/step
     ball(100%); skydiver(r)(0%)


1/1 [==============================] - 0s 30ms/step
     chess(87%); team(r)(11%)
1/1 [==============================] - 0s 20ms/step
     chess(72%); team(r)(24%)
1/1 [==============================] - 0s 30ms/step
     chess(88%); team(r)(12%)
1/1 [==============================] - 0s 30ms/step
     chess(50%); team(r)(43%)
1/1 [==============================] - 0s 30ms/step
     chess(88%); team(r)(11%)
Going through folder  day
1/1 [==============================] - 0s 30ms/step
     day(100%); skydiver(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(81%); skydiver(r)(12%)
1/1 [==============================] - 0s 30ms/step
     day(100%); tree(l)(0%)
1/1 [==============================] - 0s 30ms/step
     day(99%); tree(l)(1%)
1/1 [==============================] - 0s 30ms/step
     day(99%); team(r)(0%)
1/1 [==============================] - 0s 30ms/step
     day(99%); tree(l)(1%)
1/1 [==============================] - 0s 30ms/step
     day(100%); tree(l)(0%)

1/1 [==============================] - 0s 20ms/step
     hello(l)(80%); night(l)(15%)
1/1 [==============================] - 0s 30ms/step
     hello(l)(69%); night(l)(27%)
1/1 [==============================] - 0s 20ms/step
     hello(r)(92%); goodbye(r)(5%)
Going through folder  home
1/1 [==============================] - 0s 30ms/step
     home(60%); car(l)(27%)
1/1 [==============================] - 0s 30ms/step
     home(100%); salt(r)(0%)
1/1 [==============================] - 0s 30ms/step
     home(54%); book(30%)
1/1 [==============================] - 0s 20ms/step
     home(49%); day(28%)
1/1 [==============================] - 0s 30ms/step
     home(95%); salt(r)(4%)
1/1 [==============================] - 0s 30ms/step
     home(76%); book(16%)
1/1 [==============================] - 0s 30ms/step
     home(74%); book(14%)
1/1 [==============================] - 0s 30ms/step
     home(81%); book(14%)
1/1 [==============================] - 0s 30ms/step
     home(88%); book(7%)
1/1 [==

1/1 [==============================] - 0s 20ms/step
     only(r)(100%); interpreter(0%)
1/1 [==============================] - 0s 30ms/step
     only(r)(100%); interpreter(0%)
1/1 [==============================] - 0s 31ms/step
     only(l)(100%); tree(r)(0%)
1/1 [==============================] - 0s 21ms/step
     only(l)(99%); interpreter(1%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(l)(100%); good(l)(0%)
1/1 [==============================] - 0s 30ms/step
     only(r)(98%); chess(1%)
1/1 [==============================] - 0s 30ms/step
     chess(46%); only(r)(28%)
1/1 [==============================] - 0s 30ms/step
     only(r)(34%); chess(31%)
1/1 [==============================] - 0s 31ms/step
     only(l)(100%); interpreter(0%)
Going through folder  salt
1/1 [==============================] - 0s 30ms/step
     salt(r)(98%); hello(r)(1%)
1/1 [==============================] - 0s

1/1 [==============================] - 0s 30ms/step
     voice(l)(100%); friend(l)(0%)
1/1 [==============================] - 0s 20ms/step
     voice(l)(99%); friend(l)(1%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(93%); friend(l)(4%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(77%); friend(l)(15%)
1/1 [==============================] - 0s 30ms/step
     voice(l)(91%); friend(l)(5%)
1/1 [==============================] - 0s 30ms/step
     voice(r)(53%); friend(r)(45%)
Result:
{'apple': 8, 'ball': 10, 'bike': 10, 'book': 9, 'bowl': 8, 'boy': 7, 'brother': 10, 'butterfly': 8, 'car': 10, 'chess': 10, 'day': 10, 'evening': 10, 'father': 9, 'friend': 9, 'fruit': 8, 'good': 7, 'goodbye': 8, 'guitar': 9, 'hello': 7, 'home': 10, 'I': 10, 'interpreter': 4, 'island': 10, 'medal': 10, 'morning': 10, 'mother': 9, 'my': 3, 'night': 7, 'only': 9, 'salt': 7, 'skydiver': 9, 'spoon': 9, 'sun': 10, 'team': 9, 'thanks': 5, 'tiger': 9, 'tree': 10, 'voice': 9}


# 6. Графічний інтерфейс користувача

In [ ]:
class Application:
    def __init__(self, output_path = "./"):
        """ Initialize application which uses OpenCV + Tkinter. It displays
            a video stream in a Tkinter window and stores current snapshot on disk """
        self.cap = cv2.VideoCapture(1) 
        #######
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
        self.cap.set(cv2.CAP_PROP_FPS, frame_rate)
        #self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.frame = None
        self.curr_caption = None
        self.recognized_first = ""
        self.success, self.total = 0, 0
        #######
        self.output_path = output_path  # store output path
        self.current_image = None  # current image from the camera

        self.root = tk.Tk()  # initialize root window
        self.root.title("Распознавание жестов УЖМ")  # set window title
        # self.destructor function gets fired when the window is closed
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        # Создаем левый фрейм и размещаем на нем виджеты
        self.left_frame = tk.Frame(self.root)
        self.decode_btn = tk.Button(self.left_frame, text="Recognise!", command=lambda: threading.Thread(target=self.decode_video).start())
        self.inner1 = tk.Frame(self.left_frame)
        self.lbl1 = tk.Label(self.inner1, text="STATUS:")
        self.status = tk.Label(self.inner1, text="Ready")
        self.inner2 = tk.Frame(self.left_frame)
        self.inner3 = tk.Frame(self.left_frame)
        self.lbl2 = tk.Label(self.inner2, text="Feedback:")
        self.save_btn = tk.Button(self.inner3, text="Save video", state='disabled', command = self.save_routine)
        self.ok_btn = tk.Button(self.inner2, text="Recognised properly?", state='disabled', command = self.rec_prop_routine)
        self.stats_btn = tk.Button(self.inner3, text="Print statistics", command=lambda: self.log.insert(tk.END, f"""\nSTATISTICS:
    successfully recognised - {self.success},
    total - {self.total},
    success rate - {100*self.success/self.total if self.total else 0:3.2f}%"""))
        self.log = ScrolledText(self.left_frame, height=10, width=35)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y)
        self.decode_btn.pack(padx=5, pady=5, fill="x")
        self.inner1.pack(side=tk.TOP)
        self.lbl1.pack(side=tk.LEFT, padx=5, pady=10)
        self.status.pack(side=tk.LEFT, padx=5, pady=5)
        self.inner2.pack(side=tk.TOP)
        self.inner2.pack(side=tk.TOP)
        self.lbl2.pack(side=tk.LEFT, padx=5, pady=10)
        self.ok_btn.pack(side=tk.LEFT, padx=5, pady=5)
        self.save_btn.pack(side=tk.LEFT, padx=5, pady=5,)
        self.inner3.pack(side=tk.TOP)
        self.stats_btn.pack(padx=5, pady=5,)
        self.log.pack(fill=tk.BOTH, padx=5, pady=5, expand=True)

        # Создаем правый фрейм и размещаем на нем изображение
        self.right_frame = tk.Frame(self.root)
        self.right_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # create image panel
        self.panel = tk.Label(self.right_frame)
        self.panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # start a self.video_loop that constantly pools the video sensor
        # for the most recently read frame
        self.video_loop()

        
    def video_loop(self, caller=None):
        """ Get frame from the video stream and show it in Tkinter """
        ok, frame = self.cap.read()  # read frame from video stream
        if ok:  # frame captured without any errors
            ####
            self.frame = frame
            ####
            cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)  # convert colors from BGR to RGBA
            if not self.curr_caption:
                self.current_image = Image.fromarray(cv2image)  # convert image for PIL
            else:
                blank_image = cv2.resize(cv2.imread('blank.jpg'), (frame_width, frame_height))
                cv2.putText(blank_image, self.curr_caption, (450, int(frame_height/2)),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2, cv2.LINE_AA)
                self.current_image = Image.fromarray(blank_image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)  # convert image for tkinter
            self.panel.imgtk = imgtk  # anchor imgtk so it does not be deleted by garbage-collector
            self.panel.config(image=imgtk)  # show the image
        self.root.after(30, self.video_loop)  # call the same function after 30 milliseconds
        
    def decode_video(self):
        self.ok_btn.config(state = "disabled")
        self.save_btn.config(state = "disabled")
        self.status["text"] = "Recording sequence"
        writer = cv2.VideoWriter('./temp/temp.mp4', cv2.VideoWriter_fourcc(*'DIVX'), frame_rate, (frame_width, frame_height)) # './temp/temp.mp4'
        sequence_length = 0
        self.curr_caption = "GET READY for RECORDING"
        time.sleep(2)
        self.curr_caption = None
        start = datetime.datetime.now()
        while sequence_length < num_of_frames:
            #print("write = " + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f"))
            writer.write(cv2.resize(self.frame, (frame_width, frame_height)))
            time.sleep(.02) # ADJUST!!!
            sequence_length += 1
            #self.status["text"] = f"Recording sequence (frame {sequence_length + 1})"
        end = datetime.datetime.now()
        writer.release()
        self.curr_caption = "RECORDED SUCCESSFULLY! PLEASE WAIT!"
        self.status["text"] = f"Recognising... (duration: {(end - start).total_seconds():3.2f}s)"
        string, self.recognized_first, second = sign_from_video('./temp/temp.mp4')
        self.status["text"] = "Recognising completed!"
        self.curr_caption = ""
        self.total += 1
        self.ok_btn.config(state = "normal")
        #self.log.insert(tk.END, f"\nFirst word: {self.recognized_first}, second - {second}")
        self.log.insert(tk.END, "\n" + string)

    def rec_prop_routine(self):
        self.success += 1
        self.ok_btn.config(state = "disabled")
        self.save_btn.config(state="normal")

    def save_routine(self):
        if os.path.exists("./temp/temp.mp4"):
            os.rename("./temp/temp.mp4", f"./temp/{self.recognized_first}-{int(time.time())}.mp4")
            self.status["text"] = "Video saved!"
        self.ok_btn.config(state = "disabled")
        self.save_btn.config(state = "disabled")

    def destructor(self):
        """ Destroy the root object and release all resources """
        self.root.destroy()
        self.cap.release()  # release web camera
        cv2.destroyAllWindows()  # it is not mandatory in this application


# start the app
pba = Application()
pba.root.mainloop()

# Інші допоміжні функції

In [ ]:
# The function creates images with keypoints from the video located at `video_path`
# and saves them to `images` directory.
# `name` is the name of the output file.

def create_images_with_keypoints(video_path, name):
    
    # Create a directory to store the output images
    save_path = 'images'
    os.makedirs(os.path.join('images'), exist_ok=True)
    
    # Open the video file for reading
    cap = cv2.VideoCapture(video_path)
    
    # If the video file cannot be opened, return from the function
    if not cap.isOpened():
        return
    
    # Get the number of digits in the frame number for creating output file names
    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))
    n = 0
    # Create a Holistic object from the Mediapipe framework to detect holistic landmarks
    with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.5) as holistic:
        
        # Loop through all the frames of the video
        while True:
            
            # Read the next frame from the video file
            ret, frame = cap.read()

            # If there is a valid frame
            if ret:
                # Make detections using the Holistic model
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks on the image
                draw_styled_landmarks(image, results)
            
                # Write the image to disk with a filename formatted with the output name and frame number
                cv2.imwrite('images/{}_{}.jpg'.format(name, str(n).zfill(digit)), image)
                
                # Increment the frame number
                n += 1
            else:
                # When there are no more frames, return from the function
                return
            

def create_images_with_keypoints_v2(name, number):
    video_path = os.path.join('videos', name, '{}_{}.mp4'.format(name, number))
    keypoints_path = os.path.join('keypoints', name, '{}_{}'.format(name, number))
    imges_path = 'images'
    create_images_with_keypoints(video_path, name)
    for frame_kp in os.listdir(keypoints_path):
        if 'nouse' in frame_kp:
            n = frame_kp.split('_')[1].split('.')[0]
            os.remove(os.path.join(imges_path, '{}_{}.jpg'.format(name, n)))

            
# name - имя папки которую обработать, 
# to - cut frames to this number (default 60), 
# start_clean_at - difference between frames that are allowed to be removed
def cleanup_keyframes(name, to=60, start_clean_at=0.4):
#     path of top folder (keypoints/{name})
    dir_path = os.path.join('keypoints', name)
#     list of everything inside of the mentioned folder
    pose_list = os.listdir(dir_path)
#     go throw every entry
    for pose_num in pose_list:
#         path to pose folder (keypoints/{name}/{name}_{number})
        pose_path = os.path.join(dir_path, pose_num)
        print('Going through', pose_path)
#         list of everything inside of the mentioned folder
        frames_list = os.listdir(pose_path)
#         how many frames are in current pose
        original_frames = len(frames_list)
#         go to the next pose if frames are < then "to" param
        if original_frames <= to: continue
        
#         temp variables to store data
        prev_frame = []
        prev_frame_path = ''
        list_of_diff = []
#         go through every frame
        for frame in frames_list:
#             current frame path
            cur_frame_path = os.path.join(pose_path, frame)
#             frame data
            frame_data = np.load(cur_frame_path)
#             go to the next frame if no previous frame were saved (required for 1st frame)
            if len(prev_frame) == 0:
                prev_frame = frame_data
                prev_frame_path = cur_frame_path
                continue
            else:
                sum = 0
#                 go through current frame data and sum with previous frame
                for id, prev_point in enumerate(prev_frame):
                    cur_point = frame_data[id]
                    sum = sum + abs(prev_point - cur_point)
                
#                 save the result
                list_of_diff.append([prev_frame_path, cur_frame_path, sum, False])
                prev_frame = frame_data
                prev_frame_path = cur_frame_path
            
#         copy start_clean_at to a variable to use later
        local_treshhold = start_clean_at
        
#         go through every saved couple of frames and remove (rename) those where sum is < local_treshhold, 
#         increase local_treshhold by 0.1 if remaining frames are > "to" param
        while original_frames > to:
            for id, (f1, f2, sum, deleted) in enumerate(list_of_diff):
                if sum < local_treshhold and not deleted:
                    os.rename(f1, f1.split('.')[0] + '_nouse.npy')
                    print('\t deliting file', f1, sum, 'with treshhold of:', local_treshhold)
                    list_of_diff[id][3] = True
                    original_frames= original_frames - 1
                if original_frames <= to : break
            local_treshhold = local_treshhold + 0.1
        print('frames left:', original_frames)

In [ ]:

def get_frames_diffs(path):
    print('Going through', path)
    frames_list = os.listdir(path)

    prev_frame = []
    prev_frame_path = ''
    list_of_diff = []
    for frame in frames_list:
        cur_frame_path = os.path.join(path, frame)
        frame_data = np.load(cur_frame_path)
        if len(prev_frame) == 0:
            prev_frame = frame_data
            prev_frame_path = cur_frame_path
            continue
        else:
            sum = 0
            for id, prev_point in enumerate(prev_frame):
                cur_point = frame_data[id]
                sum = sum + abs(prev_point - cur_point)

            list_of_diff.append(sum)
            prev_frame = frame_data
            prev_frame_path = cur_frame_path

    return list_of_diff


In [ ]:
get_frames_diffs('keypoints backup\keypoints\goodbye\goodbye_01')

In [ ]:
data1 = get_frames_diffs('keypoints backup\keypoints\goodbye\goodbye_01')
data2 = get_frames_diffs('keypoints backup\keypoints\my\my_05')
data3 = get_frames_diffs('keypoints backup\keypoints\hello\hello_04')

fig, axs = plt.subplots(1)

axs.plot(range(len(data1)), data1, color='orange', markersize=20, label='до побачення')
axs.plot(range(len(data2)), data2, color='blue', markersize=20, label='мій')
axs.plot(range(len(data3)), data3, color='violet', markersize=20, label='привіт')

axs.set_xlabel('номер кадру')
axs.set_ylabel('різниця між сусідніми кадрами')
plt.legend()


# Переименовка


In [ ]:
def rename_in(root='videos'):
    for name in os.listdir(root):
        for id, vid in enumerate(os.listdir(os.path.join(root, name))):
            n = id
            while True:
                try:
                    os.rename(os.path.join(root, name, vid), os.path.join(root, name, '{}_{}.mp4'.format(name, str(n).zfill(2))))
                    break
                except:
                    n = n + 1

In [ ]:
rename_in(root='TEST_DATA')